In [ ]:
##Created by Prerit Jain

In [69]:
import importlib
import prompts.router_prompt
importlib.reload(prompts.router_prompt)
import prompts.retrieval_grade_prompt
importlib.reload(prompts.retrieval_grade_prompt)
import prompts.question_rewriter
importlib.reload(prompts.question_rewriter)
import prompts.qna_template
importlib.reload(prompts.qna_template)
import prompts.intro_gen_prompt
importlib.reload(prompts.intro_gen_prompt)

<module 'prompts.intro_gen_prompt' from 'c:\\Users\\prerit.d.jain\\OneDrive - Accenture\\JTI\\python_codes\\rfx-chatbot-demo\\prompts\\intro_gen_prompt.py'>

In [81]:
import os
from data_preprocessing import data_preprocessing
from vector_store import vector_store
from uuid import uuid4
from typing import Literal, List
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from llm_calling import llm_calling
from prompts.router_prompt import router_prompt
from prompts.retrieval_grade_prompt import retrieval_grade_prompt
from prompts.question_rewriter import question_rewriter
from prompts.qna_template import sections_create
from prompts.intro_gen_prompt import intro_gen_prompt
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.prompt import PromptTemplate
from typing_extensions import TypedDict
from langchain.schema import Document

In [ ]:
def file_names(directory):
    files=[]
    for file in os.listdir(directory):
        if file.endswith(".pdf"):
            files.append(os.path.join(directory, file))
    return files

In [ ]:
def langchain_doc_creation(file_path):
    doc=[]
    for f in file_path:
        dp = data_preprocessing(file_path=f)
        doc += dp.load_data()
    return doc

In [ ]:
directory = "./Input_Files"
file_path = file_names(directory)
print(file_path)
doc_input = langchain_doc_creation(file_path)
print(len(doc_input))
embeddings = llm_calling(embedding_model="llama3.2:latest").call_embed_model()
vs_dense = vector_store(collection_name="jti_rfp_dense",embeddings=embeddings,path="./tmp/langchain_qdrant_input").vector_qdrant_dense()
uuids = [str(uuid4()) for _ in range(len(doc_input))]
vs_dense.add_documents(documents=doc_input, ids=uuids)


In [ ]:
directory = "./User_File"
file_path = file_names(directory)
print(file_path)
doc_user = langchain_doc_creation(file_path)
print(len(doc_user))
embeddings = llm_calling(embedding_model="llama3.2:latest").call_embed_model()
vs_dense_user = vector_store(collection_name="jti_rfp_dense_user",embeddings=embeddings,path="./tmp/langchain_qdrant_user").vector_qdrant_dense()
uuids_user = [str(uuid4()) for _ in range(len(doc_user))]
vs_dense_user.add_documents(documents=doc_user, ids=uuids_user)

In [34]:
retriever_input = vs_dense.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever_user = vs_dense_user.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
llm = llm_calling(model_name="qwen2.5:7b").call_llm()

### Router

In [ ]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["universal vectorstore", "user vectorstore"] = Field(
        ...,
        description="There are two vectorstores. Given a user question choose to route it to the vectorstore for Input Files or a vectorstore for User Files.",
    )

structured_llm_router = llm.with_structured_output(RouteQuery)
system = router_prompt
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Can you tell me a little about the Evaluation criteria for the RFP?"}
    )
)
print(question_router.invoke({"question": "From the document uploaded, can you tell me about the evaluation criterial for the RFP?"}))

### Retrieval Accuracy

In [35]:
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = retrieval_grade_prompt
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [61]:
question = "Create an Intoduction section for the RFP document, giving a bried overview on JTI (Japan Tobacco International)."
docs = retriever_input.invoke(question)
doc_txt=''
for d in docs:
    doc_txt+= d.page_content
    doc_txt+= "\n\n"

In [42]:
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [43]:
question = "What is evaluation criteria for the RFP?"
docs = retriever_user.invoke(question)
doc_txt=''
for d in docs:
    doc_txt+= d.page_content
    doc_txt+= "\n\n"

In [44]:
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


### Question Rewriter

In [ ]:
### Question Re-writer

# Prompt
system = question_rewriter
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human","Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'What are the key evaluation criteria specified in the RFP?'

### Section Creator

In [75]:
prompt=PromptTemplate(template=sections_create, input_variables=['question','context'])
question="Create an Intoduction section for the RFP document, giving a brief overview on JTI (Japan Tobacco International)."

In [62]:

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()
docs = format_docs(docs)

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Certainly! Here's an introduction section for the RFP document, providing a brief overview of JTI (Japan Tobacco International):

---

**Introduction**

Japan Tobacco International (JTI) is a global leader in the tobacco and related products industry. With a strong presence in numerous markets worldwide, JTI continues to innovate and expand its portfolio to meet evolving consumer needs while adhering to regulatory standards. As part of our ongoing efforts to enhance customer engagement and loyalty, we are seeking proposals for a digital reward solution that can seamlessly integrate with existing platforms and provide an engaging user experience.

We invite you to join us in this exciting endeavor by leveraging your expertise to develop a robust and user-friendly digital reward system tailored to the JIBY brand. Together, we aim to build a successful partnership that will not only meet but exceed our expectations for a seamless user journey, simplicity, and engagement.

---

Feel free t

In [64]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [66]:
def retrieve_universal(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever_input.invoke(question)
    return {"documents": documents, "question": question}

def retrieve_user(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever_user.invoke(question)
    return {"documents": documents, "question": question}

In [76]:
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

In [77]:
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

In [78]:
def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

In [ ]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "universal vectorstore":
        print("---ROUTE QUESTION TO UNIVERSAL VECTORSTORE---")
        return "universal vectorstore"
    elif source.datasource == "user vectorstore":
        print("---ROUTE QUESTION TO USER VECTORSTORE---")
        return "user vectorstore"